<a href="https://colab.research.google.com/github/MayarAtef16/Reinforcement-to-learn-promising-paths-in-robot-style-grid-world./blob/main/AI_%7C_MiniProject2_Final_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Implementation

In [ ]:
from shutil import get_terminal_size
terminal_width, _ = get_terminal_size()

_visualizers = {}

def _default_visualizer(_, state):
    '''Generic visualizer for unknown problems.'''
    print(state)

class Visualizer:
    '''Visualization and printing functionality encapsulation.'''

    def __init__(self, problem):
        '''Constructor with the problem to visualize.'''
        self.problem = problem
        self.counter = 0

    def visualize(self, frontier):
        '''Visualizes the frontier at every step.'''
        self.counter += 1
        print(f'Frontier at step {self.counter}')
        for state in frontier:
            print()
            _visualizers.get(type(self.problem), _default_visualizer)(self.problem, state)
        print('-' * terminal_width)

    #self,Agent, pickup_locations, delivery_locations, bounds, max_reward, discount
def _ZC_agent_visualizer(env, state):
  '''Custom Visualizer for ZC agent'''
  Agent,pickup_locations,delivery_locations = env.state
  for j in range(env.bounds[1] - 1, -1, -1):
    for i in range(env.bounds[0]):
      print('🤖' if (i, j) == Agent else '📦' if (i, j) == pickup_locations else '🚚' if (i, j) == delivery_locations else '⬜', end='')
    print()

In [ ]:
class Environment:
    '''
    Abstract base class for an (interactive) environment formulation.
    It declares the expected methods to be used to solve it.
    All the methods declared are just placeholders that throw errors if not overriden by child "concrete" classes!
    '''

    def __init__(self):
        '''Constructor that initializes the problem. Typically used to setup the initial state.'''
        self.state = None

    def actions(self):
        '''Returns an iterable with the applicable actions to the current environment state.'''
        raise NotImplementedError

    def apply(self, action):
        '''Applies the action to the current state of the environment and returns the new state from applying the given action to the current environment state; not necessarily deterministic.'''
        raise NotImplementedError

    @classmethod
    def new_random_instance(cls):
        '''Factory method to a problem instance with a random initial state.'''
        raise NotImplementedError


#ZC Enviroment

In [ ]:
from random import choice, randrange

class ZC(Environment):

    def __init__(self,Agent, pickup_locations, delivery_locations, bounds, max_reward, discount=0.3):
        self.items = []  # Initialize the items the agent is carrying to an empty list
        self.pickup_locations = pickup_locations  # Initialize the pickup locations and the number of items at each location
        self.delivery_locations = delivery_locations  # Initialize the delivery locations and the number of items at each location
        self.state = (Agent, tuple(pickup_locations.values()),tuple(delivery_locations.values()) )  # Initialize the state of the agent at the main gate of ZC'
        self.bounds = bounds
        self.max_reward = max_reward
        self.discount = discount

    def actions(self):
      if self.state is not None:
        if self.state[0] in self.pickup_locations :
          if self.pickup_locations[self.state[0]] > 0:
            if len(self.items) == 0:
             return ['pickup']
        if self.state[0] in self.delivery_locations :
          if self.delivery_locations[self.state[0]] < 4:
           if len(self.items) > 0:
             return ['deliver']
      return ['up', 'down', 'left', 'right']


    def apply(self, action):  #applies the action & gives reward.
           #print(self.pickup_locations.values())
           #print(self.delivery_locations.values())

           if all(value == 0 for value in self.pickup_locations.values()) and all(value == 4 for value in self.delivery_locations.values()):
               self.state = None
               print('done')
               return 0  # Return a reward of 0 and the current state

           # Define the functions for the basic movements
           up = lambda position: (position[0], max(position[1] - 1, 0))
           down = lambda position: (position[0], min(position[1] + 1, self.bounds[1] - 1))
           left = lambda position: (max(position[0] - 1, 0), position[1])
           right = lambda position: (min(position[0] + 1, self.bounds[0] - 1), position[1])

           #x, y = self.state
           if action == 'left':
            self.state = (left(self.state[0]), tuple(self.pickup_locations.values()) ,tuple(self.delivery_locations.values()))
            #print('left',self.state[0])
            #return -1 if len(self.items) == 0 else 1  # Return a positive reward for moving with items, a negative reward for moving with no items
            return -1
            #if len(self.items) == 0:return -1 if len(self.items) == 0
           elif action == 'right':
            self.state = (right(self.state[0]), tuple(self.pickup_locations.values()) ,tuple(self.delivery_locations.values()))
            #print('right',self.state[0])
            #return -1 if len(self.items) == 0 else 1  # Return a positive reward for moving with items, a negative reward for moving with no items
            return -1
           elif action == 'up':
            self.state = (up(self.state[0]), tuple(self.pickup_locations.values()) ,tuple(self.delivery_locations.values()))
            #print('up',self.state[0])
            #return -1 if len(self.items) == 0 else 1  # Return a positive reward for moving with items, a negative reward for moving with no items
            return -1
           elif action == 'down':
            self.state = (down(self.state[0]), tuple(self.pickup_locations.values()) ,tuple(self.delivery_locations.values()))
            #print('down',self.state[0])
            #return -1 if len(self.items) == 0 else 1  # Return a positive reward for moving with items, a negative reward for moving with no items
            return -1
           elif action == 'pickup':
               #print('pickup',self.state[0])
               if self.state[0] in self.pickup_locations and self.pickup_locations[self.state[0]] > 0:  # If the agent is at a pickup location and there are items available to pick up
                  if len(self.items) == 0:
                   self.items.append(1)  # Pick up the item
                   #print(len(self.items))
                   #print(self.state)
                   self.pickup_locations[self.state[0]] -= 1  # Decrement the number of items at the pickup location
                   #print('pickup',reward)
                   self.state = (self.state[0], tuple(self.pickup_locations.values()) ,tuple(self.delivery_locations.values()) )
                   print('/pickup\, state, pickup value',self.state,self.pickup_locations)
                   #print(self.state)
                   return self.max_reward
           elif action == 'deliver':
               #print('deliver',self.state[0])
               # If the agent is at a delivery location and is carrying an item
               if self.state[0] in self.delivery_locations and len(self.items) > 0:
                 if len(self.items) == 1:
                   # Increment the number of items at the delivery location
                   self.delivery_locations[self.state[0]] += 1
                   # Remove the item from the agent's inventory
                   self.items.pop()
                   #print(len(self.items))
                   # Calculate the reward based on the number of items delivered
                   #reward = self.max_reward * self.delivery_locations[(x, y)]
                   self.state = (self.state[0], tuple(self.pickup_locations.values()) ,tuple(self.delivery_locations.values()) )
                   print('/deliver\, state, deliver value',self.state,self.delivery_locations)
                   #print(self.state)
                   return self.max_reward
           return 0 # Return a reward of 0 and the current state


    @classmethod
    def new_random_instance(cls, bounds, max_reward, Lrate, discount):
        random_pair = lambda: (randrange(bounds[0]), randrange(bounds[1]), randrange(bounds[2]))
        return cls(random_pair(), random_pair(), random_pair(), bounds, max_reward, Lrate, discount)

_visualizers[ZC] = _ZC_agent_visualizer


#Q-learning

- the alpha parameter is a function that takes the number of times the action has been taken (n) and returns the learning rate. so we can modify the function to use a fixed learning rate.

In [ ]:
def q_learning(env, q={}, n={}, f=lambda q, n: (q+1)/(n+1), alpha=0.5, error=1e-6, verbose=False):
    '''Q-learning implementation that trains on an environment till no more actions can be taken'''
    if verbose: visualizer = Visualizer(env)
    while env.state is not None:
        if verbose: visualizer.visualize([env.state])
        state = env.state
        action = max(env.actions(),
                     key=lambda next_action: f(q.get((state, next_action), 0), n.get((state, next_action), 0)))
        n[(state, action)] = n.get((state, action), 0) + 1
        reward = env.apply(action)
        q[(state, action)] = q.get((state, action), 0) + alpha * (reward + env.discount * max((q.get((env.state, next_action), 0) for next_action in env.actions()), default=0) - q.get((state, action), 0))
    return q, n


In [ ]:
from math import inf
from time import time
from itertools import count

def simulate(env_ctor, n_iterations=inf, duration=inf, **q_learning_params):
    '''A helper function to train for a fixed number of iterations or fixed time'''
    for param in ('q', 'n'): q_learning_params[param] = q_learning_params.get(param, {})
    start_time = time()
    i = count()
    while time() < start_time + duration and next(i) < n_iterations:
        env = env_ctor()
        q, n = q_learning(env, **q_learning_params)
    return q_learning_params['q'], q_learning_params['n']

In [ ]:
simulate(lambda: ZC((0,0), {(4,1):4, (4,2):4}, {(6,1):0, (6,2):0}, (25,25), 1000, 0.3), n_iterations = 3000, verbose=False)

Streaming output truncated to the last 5000 lines.
/deliver\, state, deliver value ((6, 2), (0, 0), (4, 4)) {(6, 1): 4, (6, 2): 4}
done
/pickup\, state, pickup value ((4, 1), (3, 4), (0, 0)) {(4, 1): 3, (4, 2): 4}
/deliver\, state, deliver value ((6, 1), (3, 4), (1, 0)) {(6, 1): 1, (6, 2): 0}
/pickup\, state, pickup value ((4, 1), (2, 4), (1, 0)) {(4, 1): 2, (4, 2): 4}
/deliver\, state, deliver value ((6, 1), (2, 4), (2, 0)) {(6, 1): 2, (6, 2): 0}
/pickup\, state, pickup value ((4, 1), (1, 4), (2, 0)) {(4, 1): 1, (4, 2): 4}
/deliver\, state, deliver value ((6, 1), (1, 4), (3, 0)) {(6, 1): 3, (6, 2): 0}
/pickup\, state, pickup value ((4, 1), (0, 4), (3, 0)) {(4, 1): 0, (4, 2): 4}
/deliver\, state, deliver value ((6, 1), (0, 4), (4, 0)) {(6, 1): 4, (6, 2): 0}
/pickup\, state, pickup value ((4, 2), (0, 3), (4, 0)) {(4, 1): 0, (4, 2): 3}
/deliver\, state, deliver value ((6, 2), (0, 3), (4, 1)) {(6, 1): 4, (6, 2): 1}
/pickup\, state, pickup value ((4, 2), (0, 2), (4, 1)) {(4, 1): 0, (4, 2):

({(((0, 0), (4, 4), (0, 0)), 'up'): -1.1735625,
  (((0, 0), (4, 4), (0, 0)), 'down'): -1.2779375,
  (((0, 1), (4, 4), (0, 0)), 'up'): -1.210003125,
  (((0, 0), (4, 4), (0, 0)), 'left'): -1.1735625,
  (((0, 0), (4, 4), (0, 0)), 'right'): 1.071356628982032,
  (((1, 0), (4, 4), (0, 0)), 'up'): -1.1735625,
  (((1, 0), (4, 4), (0, 0)), 'down'): -1.2059375,
  (((1, 1), (4, 4), (0, 0)), 'up'): -1.0566875,
  (((1, 0), (4, 4), (0, 0)), 'left'): -1.129484375,
  (((0, 1), (4, 4), (0, 0)), 'down'): -1.173125,
  (((0, 2), (4, 4), (0, 0)), 'up'): -1.06625,
  (((0, 1), (4, 4), (0, 0)), 'left'): -1.1735625,
  (((0, 1), (4, 4), (0, 0)), 'right'): -1.1491875,
  (((1, 1), (4, 4), (0, 0)), 'down'): -1.104375,
  (((1, 2), (4, 4), (0, 0)), 'up'): -1.0566875,
  (((1, 1), (4, 4), (0, 0)), 'left'): -1.1110625,
  (((1, 0), (4, 4), (0, 0)), 'right'): 6.904522096606774,
  (((2, 0), (4, 4), (0, 0)), 'up'): -1.03625,
  (((2, 0), (4, 4), (0, 0)), 'down'): 26.34840698868925,
  (((2, 1), (4, 4), (0, 0)), 'up'): -0.825

### Greedy Q-Learning

In [ ]:
# GREEDY Q_LEARNING  / New random instance needs to be fixed { the input to it}
q, n = {}, {}
simulate(lambda: ZC( (0,0) , {(4,1):4,(4,2):4}, {(2,1):0,(3,2):0} , (7,7), 1000, 0.3), duration=1, q=q, n=n, f=lambda q, n: q)
simulate(lambda: ZC( (0,0) , {(4,1):4,(4,2):4}, {(2,1):0,(3,2):0} , (7,7), 1000, 0.3), n_iterations=3000, q=q, n=n, f=lambda q, n: q)

Streaming output truncated to the last 5000 lines.
/deliver\, state, deliver value ((3, 2), (3, 3), (0, 2)) {(2, 1): 0, (3, 2): 2}
/pickup\, state, pickup value ((4, 2), (3, 2), (0, 2)) {(4, 1): 3, (4, 2): 2}
/deliver\, state, deliver value ((3, 2), (3, 2), (0, 3)) {(2, 1): 0, (3, 2): 3}
/pickup\, state, pickup value ((4, 2), (3, 1), (0, 3)) {(4, 1): 3, (4, 2): 1}
/deliver\, state, deliver value ((3, 2), (3, 1), (0, 4)) {(2, 1): 0, (3, 2): 4}
/pickup\, state, pickup value ((4, 2), (3, 0), (0, 4)) {(4, 1): 3, (4, 2): 0}
/deliver\, state, deliver value ((2, 1), (3, 0), (1, 4)) {(2, 1): 1, (3, 2): 4}
/pickup\, state, pickup value ((4, 1), (2, 0), (1, 4)) {(4, 1): 2, (4, 2): 0}
/deliver\, state, deliver value ((2, 1), (2, 0), (2, 4)) {(2, 1): 2, (3, 2): 4}
/pickup\, state, pickup value ((4, 1), (1, 0), (2, 4)) {(4, 1): 1, (4, 2): 0}
/deliver\, state, deliver value ((2, 1), (1, 0), (3, 4)) {(2, 1): 3, (3, 2): 4}
/pickup\, state, pickup value ((4, 1), (0, 0), (3, 4)) {(4, 1): 0, (4, 2): 0}
/

### Random Q-Learning

In [ ]:
from random import random
q, n = {}, {}
simulate(lambda: ZC( (0,0) , {(0,1):4,(0,2):4}, {(1,1):0,(1,2):0} , (7,7), 1000, 0.3), duration=1, q=q, n=n, f=lambda q, n: random())
simulate(lambda: ZC( (0,0) , {(0,1):4,(0,2):4}, {(1,1):0,(1,2):0} , (7,7), 1000, 0.3), n_iterations=3000, q=q, n=n, verbose=False, f=lambda q, n: q)

Streaming output truncated to the last 5000 lines.
/deliver\, state, deliver value ((1, 1), (0, 0), (4, 4)) {(1, 1): 4, (1, 2): 4}
done
/pickup\, state, pickup value ((0, 1), (3, 4), (0, 0)) {(0, 1): 3, (0, 2): 4}
/deliver\, state, deliver value ((1, 1), (3, 4), (1, 0)) {(1, 1): 1, (1, 2): 0}
/pickup\, state, pickup value ((0, 1), (2, 4), (1, 0)) {(0, 1): 2, (0, 2): 4}
/deliver\, state, deliver value ((1, 1), (2, 4), (2, 0)) {(1, 1): 2, (1, 2): 0}
/pickup\, state, pickup value ((0, 2), (2, 3), (2, 0)) {(0, 1): 2, (0, 2): 3}
/deliver\, state, deliver value ((1, 2), (2, 3), (2, 1)) {(1, 1): 2, (1, 2): 1}
/pickup\, state, pickup value ((0, 2), (2, 2), (2, 1)) {(0, 1): 2, (0, 2): 2}
/deliver\, state, deliver value ((1, 2), (2, 2), (2, 2)) {(1, 1): 2, (1, 2): 2}
/pickup\, state, pickup value ((0, 2), (2, 1), (2, 2)) {(0, 1): 2, (0, 2): 1}
/deliver\, state, deliver value ((1, 2), (2, 1), (2, 3)) {(1, 1): 2, (1, 2): 3}
/pickup\, state, pickup value ((0, 2), (2, 0), (2, 3)) {(0, 1): 2, (0, 2):

({(((0, 0), (4, 4), (0, 0)), 'right'): 27.452530888428704,
  (((1, 0), (4, 4), (0, 0)), 'right'): 3.325548409952636,
  (((2, 0), (4, 4), (0, 0)), 'left'): 22.313836846037102,
  (((1, 0), (4, 4), (0, 0)), 'left'): 96.55750597157211,
  (((0, 0), (4, 4), (0, 0)), 'left'): 96.21589696176696,
  (((0, 0), (4, 4), (0, 0)), 'down'): 328.5562726614089,
  (((0, 1), (4, 4), (0, 0)), 'pickup'): 1098.5209088713632,
  (((0, 1), (3, 4), (0, 0)), 'left'): 93.5011957347287,
  (((0, 1), (3, 4), (0, 0)), 'right'): 328.40302957121105,
  (((1, 1), (3, 4), (0, 0)), 'deliver'): 1098.0100985707036,
  (((1, 1), (3, 4), (1, 0)), 'left'): 326.7003285690121,
  (((0, 1), (3, 4), (1, 0)), 'pickup'): 1092.3344285633739,
  (((0, 1), (2, 4), (1, 0)), 'down'): 53.867062499999996,
  (((0, 2), (2, 4), (1, 0)), 'down'): -0.9375,
  (((0, 3), (2, 4), (1, 0)), 'left'): -0.875,
  (((0, 3), (2, 4), (1, 0)), 'up'): 21.4,
  (((0, 2), (2, 4), (1, 0)), 'up'): 7.330059374999999,
  (((0, 1), (2, 4), (1, 0)), 'left'): 7.2050593749999

### Exploring Q-Learning

In [ ]:
q, n = {}, {}
simulate(lambda: ZC( (0,0) , {(0,1):4,(0,2):4}, {(1,1):0,(1,2):0} , (7,7), 1000, 0.3), duration=1, q=q, n=n, f=lambda q, n: 1/(n+1))
simulate(lambda: ZC( (0,0) , {(0,1):4,(0,2):4}, {(1,1):0,(1,2):0} , (7,7), 1000, 0.3), n_iterations=3000, q=q, n=n, verbose=False, f=lambda q, n: q)

Streaming output truncated to the last 5000 lines.
/deliver\, state, deliver value ((1, 2), (0, 0), (4, 4)) {(1, 1): 4, (1, 2): 4}
done
/pickup\, state, pickup value ((0, 1), (3, 4), (0, 0)) {(0, 1): 3, (0, 2): 4}
/deliver\, state, deliver value ((1, 1), (3, 4), (1, 0)) {(1, 1): 1, (1, 2): 0}
/pickup\, state, pickup value ((0, 1), (2, 4), (1, 0)) {(0, 1): 2, (0, 2): 4}
/deliver\, state, deliver value ((1, 1), (2, 4), (2, 0)) {(1, 1): 2, (1, 2): 0}
/pickup\, state, pickup value ((0, 1), (1, 4), (2, 0)) {(0, 1): 1, (0, 2): 4}
/deliver\, state, deliver value ((1, 1), (1, 4), (3, 0)) {(1, 1): 3, (1, 2): 0}
/pickup\, state, pickup value ((0, 1), (0, 4), (3, 0)) {(0, 1): 0, (0, 2): 4}
/deliver\, state, deliver value ((1, 1), (0, 4), (4, 0)) {(1, 1): 4, (1, 2): 0}
/pickup\, state, pickup value ((0, 2), (0, 3), (4, 0)) {(0, 1): 0, (0, 2): 3}
/deliver\, state, deliver value ((1, 2), (0, 3), (4, 1)) {(1, 1): 4, (1, 2): 1}
/pickup\, state, pickup value ((0, 2), (0, 2), (4, 1)) {(0, 1): 0, (0, 2):

({(((0, 0), (4, 4), (0, 0)), 'up'): 97.42921529013864,
  (((0, 0), (4, 4), (0, 0)), 'down'): 328.57142757704923,
  (((0, 1), (4, 4), (0, 0)), 'pickup'): 1098.571425256831,
  (((0, 1), (3, 4), (0, 0)), 'up'): 27.110266980000688,
  (((0, 0), (3, 4), (0, 0)), 'up'): 25.922279940302815,
  (((0, 0), (3, 4), (0, 0)), 'down'): 95.44451403958968,
  (((0, 1), (3, 4), (0, 0)), 'down'): 83.11334073852538,
  (((0, 2), (3, 4), (0, 0)), 'up'): 92.64849176002502,
  (((0, 1), (3, 4), (0, 0)), 'left'): 96.55714188044311,
  (((0, 1), (3, 4), (0, 0)), 'right'): 328.57141752277005,
  (((1, 1), (3, 4), (0, 0)), 'deliver'): 1098.571391742567,
  (((1, 1), (3, 4), (1, 0)), 'up'): 23.18153437495117,
  (((1, 0), (3, 4), (1, 0)), 'up'): 19.66981211816406,
  (((1, 0), (3, 4), (1, 0)), 'down'): 87.88438674121093,
  (((1, 1), (3, 4), (1, 0)), 'down'): 65.24474604492188,
  (((1, 2), (3, 4), (1, 0)), 'up'): 79.17785798828126,
  (((1, 1), (3, 4), (1, 0)), 'left'): 328.5713058085573,
  (((0, 1), (3, 4), (1, 0)), 'picku